### PTB-XL 데이터 수집 단계 안내

#### 1. wfdb 패키지 설치.

pip install wfdb 명령어를 통해서 wfdb 패키지를 설치해주세요. (PTB-XL 전처리를 위하여 필요한 패키지 입니다.)

wfdb 패키지의 간단한 사용 방법이 있는 노트북 공유 드립니다.
- https://github.com/MIT-LCP/wfdb-python/blob/main/demo.ipynb


#### 2. 전처리에 필요한 패키지 import

In [1]:
import ast
import wfdb 
import numpy as np
import pandas as pd

from glob import glob 
from tqdm import tqdm

#### 3. metadata 읽기

다운로드 받은 PTB-XL 데이터에는 ptbxl_database.csv 파일이 있습니다.

해당 csv 파일을 pandas를 이용하여 열면, 맨 마지막 두개의 컬럼 filename_lr, filename_hr이 있습니다.

filename_lr은 100Hz로 저장되어 있는 신호 데이터 파일 경로이고, filename_hr은 500Hz로 저장되어 있는 신호 데이터 파일 경로입니다.

저희는 500Hz로 수집된 데이터를 사용할 것이기 때문에 filename_hr 컬럼의 경로를 이용하여 데이터를 읽으면 됩니다.

In [2]:
path = '/DATA4/afib-renew/raw/PTB-XL/'
df = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')

In [3]:
df.head()

,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
ecg_id,,,,,,,,,,,,,,,,,,,,,
1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr
4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,...,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr
5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,...,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr


#### 4. 신호 데이터 읽기

하나의 샘플을 읽어보겠습니다.

In [4]:
filenames = df.filename_hr.tolist()
base_path = '/DATA4/afib-renew/raw/PTB-XL/'
signal, meta = wfdb.rdsamp(base_path + filenames[0])

데이터를 읽으면 signal 변수에 신호 데이터가, meta 변수에는 여러가지 신호에 대한 정보 데이터가 담겨져 있습니다.

아래와 같이 signal 변수를 프린트 해보면 numpy array로 저장이 되어 있고, shape은 5000x12 형태인 것을 확인할 수 있습니다.

In [5]:
print(signal)
print(signal.shape)

[[-0.115 -0.05   0.065 ... -0.035 -0.035 -0.075]
 [-0.115 -0.05   0.065 ... -0.035 -0.035 -0.075]
 [-0.115 -0.05   0.065 ... -0.035 -0.035 -0.075]
 ...
 [ 0.21   0.205 -0.005 ...  0.185  0.17   0.18 ]
 [ 0.21   0.205 -0.005 ...  0.185  0.17   0.18 ]
 [ 0.21   0.205 -0.005 ...  0.185  0.17   0.18 ]]
(5000, 12)


OT 때 잠깐 말씀 드렸듯이, ECG는 측정하는 위치에 따라 총 12개의 신호를 담고 있습니다.

12개의 Lead 데이터라고 불리는데, 각 Lead의 이름은 다음과 같습니다.
- 'I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'

이 가운데, 우리는 Lead I 을 사용할 것입니다. 이는 갤럭시 워치나 애플 워치와 같이 두 팔에서 수집된 신호 데이터 입니다.

따라서, 5000x12 shape의 array에서 Lead I에 해당되는 5000개의 데이터만 저장하면 됩니다. 

예제는 다음과 같습니다.

In [6]:
lead1 = signal[:, 0]

위와 같이 0번째 인덱스의 5000개 신호만 인덱싱을 해서 신호값을 저장하면 됩니다.

나머지 신호 데이터는 사용하지 않습니다.

위와 같은 방식으로, 모든 데이터를 읽어서 Lead 1 신호를 저장하여 하나의 array로 담아 저장하시면 됩니다.

최종 저장 데이터 array의 shape은 (N, 5000)이 될 것입니다.

#### 5. Labeling 방법 안내

맨 처음에 읽었던 ptbxl_database.csv 파일로 돌아가면, 해당 dataframe에는 scp_codes 라는 컬럼이 있습니다.

In [7]:
df.scp_codes

ecg_id
1                 {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
2                             {'NORM': 80.0, 'SBRAD': 0.0}
3                               {'NORM': 100.0, 'SR': 0.0}
4                               {'NORM': 100.0, 'SR': 0.0}
5                               {'NORM': 100.0, 'SR': 0.0}
                               ...                        
21833    {'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...
21834             {'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
21835                           {'ISCAS': 50.0, 'SR': 0.0}
21836                           {'NORM': 100.0, 'SR': 0.0}
21837                           {'NORM': 100.0, 'SR': 0.0}
Name: scp_codes, Length: 21799, dtype: object

해당 컬럼의 값은 string 형태로 저장되어 있으며, string은 Python의 dictionary 형태로 되어 있는 것을 확인할 수 있습니다.

따라서, string으로 저장되어 있는 dictionary 형태의 데이터를 실제 dictionary 값으로 변환해주는 작업이 필요합니다.

예제는 아래와 같습니다.

In [8]:
df.scp_codes = df.scp_codes.apply(lambda x: ast.literal_eval(x))

위 명령어를 실행하면 scp_codes 컬럼의 각 row는 실제 dictionary 값으로 변환이 됩니다.

In [9]:
df.scp_codes

ecg_id
1                 {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
2                             {'NORM': 80.0, 'SBRAD': 0.0}
3                               {'NORM': 100.0, 'SR': 0.0}
4                               {'NORM': 100.0, 'SR': 0.0}
5                               {'NORM': 100.0, 'SR': 0.0}
                               ...                        
21833    {'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...
21834             {'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
21835                           {'ISCAS': 50.0, 'SR': 0.0}
21836                           {'NORM': 100.0, 'SR': 0.0}
21837                           {'NORM': 100.0, 'SR': 0.0}
Name: scp_codes, Length: 21799, dtype: object

해당 dictionary 값들 가운데, 우리의 관심 대상은 key 값 입니다.

key에 해당하는 것이 실제 해당 ECG 데이터의 label 정보를 담고 있습니다.

따라서, 아래와 같이 각 dictionary의 key 값을 추출하여, 다시 scp_codes 컬럼에 저장합니다.

In [10]:
df.scp_codes = df.scp_codes.apply(lambda x: list(x.keys()))

위 코드를 실행하면 아래와 같이, dictionary의 key 값만 추출하여 list에 저장한 결과를 볼 수 있습니다.

In [11]:
df.scp_codes

ecg_id
1               [NORM, LVOLT, SR]
2                   [NORM, SBRAD]
3                      [NORM, SR]
4                      [NORM, SR]
5                      [NORM, SR]
                   ...           
21833    [NDT, PVC, VCLVH, STACH]
21834           [NORM, ABQRS, SR]
21835                 [ISCAS, SR]
21836                  [NORM, SR]
21837                  [NORM, SR]
Name: scp_codes, Length: 21799, dtype: object

마지막 단계입니다.

이제 scp_codes 컬럼의 각 row 마다 저장되어 있는 list에 'AFIB' 이라는 단어가 있는 지 확인하는 단계입니다.

만약 list에 AFIB 이라는 단어가 있으면 1, 없으면 0으로 저장하면 됩니다.

아래는 해당 코드입니다.

In [12]:
df['label'] = df.scp_codes.apply(lambda arr: 1 if 'AFIB' in arr else 0)

성공적으로 labeling을 마쳤습니다.

한번, labeling된 데이터의 개수를 출력해보겠습니다.

In [13]:
df.label.value_counts()

label
0    20285
1     1514
Name: count, dtype: int64

Non-AFIB이 20,285개, AFIB이 1,514개로 출력된 것을 확인하였다면, 제대로 labeling이 된 것입니다.

#### 6. 데이터 저장

추출한 신호 데이터와 정답 데이터를 팀원분들께서 사용하시기 편한 형태로 저장합니다.

In [14]:
# 데이터 저장 샘플 코드 (다른 데이터 형태로 저장하셔도 무방합니다. 편한 방법을 선택하세요.)
# np.save('/DATA4/afib-renew/processed/PTB-XL-X.npy', X)
# np.save('/DATA4/afib-renew/processed/PTB-XL-Y.npy', Y)